In [22]:
from pettingzoo.classic import connect_four_v3
import random
import numpy as np
from tqdm import tqdm
import time

In [23]:
q_table = {}
def create_q_table_entry(state):
    name = "".join([str(x) for x in state.flatten()])
    if name not in q_table.keys():
        q_table[name] = [0 for _ in range(7)]
    return name

env = connect_four_v3.env()
epsilon = 0.2
alpha = 0.1
gamma = 0.7

for _ in tqdm(range(100000)):

    env.reset()
    agents = {
        0: {"name":"player_0","last_state": None, "current_state": None, "reward": 0},
        1: {"name":"player_1","last_state": None, "current_state": None, "reward": 0},
    }
    end = False

    i = 0

    while end is False:

        current_agent = agents[i%2]["name"]
        env.agent_selection = current_agent

        state = env.observe(current_agent)
        key=create_q_table_entry(state["observation"])

        agents[i%2]["last_state"]=key
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space(current_agent).sample(
                state["action_mask"]
            )
        else:
            possible = [
                q_table[key][i] if state["action_mask"][i] != 0 else -np.inf for i in range(7)
            ]
            action = np.argmax(possible)

        
        env.step(action)
        state, reward, termination, truncation, info = env.last()

        agents[i%2]["reward"]=reward

        key=create_q_table_entry(state["observation"])
        agents[i%2]["current_state"]=key


        end = termination or truncation

        if end:
            if agents[i%2]["reward"] == 1:
                agents[(i+1)%2]["reward"]=-1

            for j in [0,1]:
                old_value = q_table[agents[j]["last_state"]][action]
                next_max = np.max(q_table[agents[j]["current_state"]])
                new_value = (1 - alpha) * old_value + alpha * (agents[j]["reward"] + gamma * next_max)
                q_table[agents[j]["last_state"]][action] = new_value

        elif agents[(i+1)%2]["last_state"]!=None :
            old_value = q_table[agents[(i+1)%2]["last_state"]][action]
            next_max = np.max(q_table[agents[(i+1)%2]["current_state"]])
            new_value = (1 - alpha) * old_value + alpha * (agents[(i+1)%2]["reward"] + gamma * next_max)
            q_table[agents[(i+1)%2]["last_state"]][action] = new_value
        i+=1

        env.close()

100%|██████████| 100000/100000 [09:54<00:00, 168.09it/s]


In [25]:
env = connect_four_v3.env(render_mode="human")
env.reset()

agents = {
    0: {"name":"player_0","last_state": None, "current_state": None, "reward": 0},
    1: {"name":"player_1","last_state": None, "current_state": None, "reward": 0},
}
end = False
epsilon = 0.1
alpha = 0.1
gamma = 0.7

i = 0


while end is False:
    current_agent = agents[i%2]["name"]
    env.agent_selection = current_agent

    state = env.observe(current_agent)
    key=create_q_table_entry(state["observation"])

    agents[i%2]["last_state"]=key
    
    if random.uniform(0, 1) < 0:
        action = env.action_space(current_agent).sample(
            state["action_mask"]
        )
    else:
        
        possible = [
            q_table[key][i] if state["action_mask"][i] != 0 else -np.inf for i in range(7)
        ]

        action = np.argmax(possible)

    
    env.step(action)
    state, reward, termination, truncation, info = env.last()

    agents[i%2]["reward"]=reward

    key=create_q_table_entry(state["observation"])
    agents[i%2]["current_state"]=key


    end = termination or truncation

    i+=1
    time.sleep(0.3)
    

env.close()